In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

import sys
import os
import pytreebank
import nltk
import re


from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import f1_score, accuracy_score
from textblob import TextBlob
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
import string

from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.python.keras.callbacks import EarlyStopping

### IMDB DATASET

In [2]:
csv_file_path = os.path.join('..', '..', 'data', 'IMDB Dataset.csv')

if os.path.exists(csv_file_path):
    df = pd.read_csv(csv_file_path)
    print(df.head())
else:           
    print("CSV file not found.")

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [3]:
sentiment_mapping = {'positive': 1, 'negative': 0}
df['sentiment_number'] = df['sentiment'].map(sentiment_mapping)

In [4]:
df

,review,sentiment,sentiment_number
0,One of the other reviewers has mentioned that ...,positive,1
1,A wonderful little production. <br /><br />The...,positive,1
2,I thought this was a wonderful way to spend ti...,positive,1
3,Basically there's a family where a little boy ...,negative,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1
...,...,...,...
49995,I thought this movie did a down right good job...,positive,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative,0
49997,I am a Catholic taught in parochial elementary...,negative,0
49998,I'm going to have to disagree with the previou...,negative,0


In [9]:
#Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def preprocess_text(text):
    text = text.lower()
    return text

def remove_punctuation(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    filtered_text = [word for word in text.split() if word not in stop_words]
    return " ".join(filtered_text)


In [8]:
df['review']=df['review'].apply(strip_html)
df['review']=df['review'].apply(preprocess_text)
df['review']=df['review'].apply(remove_punctuation)
df['review']=df['review'].apply(remove_stopwords)

In [6]:
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)

# Print the shapes of the train and test DataFrames
print("Train Data:")
print("Shape:", train_df.shape)

print("Test Data:")
print("Shape:", test_df.shape)

Train Data:
Shape: (35000, 3)
Test Data:
Shape: (15000, 3)


In [9]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_df['review'], train_df['sentiment_number']))
test_dataset = tf.data.Dataset.from_tensor_slices((test_df['review'], test_df['sentiment_number']))

#### Google News Corpus


In [7]:
embedding = "https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1"

hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)

model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))
model.summary()

model.compile(optimizer='adam', 
loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
metrics=['accuracy'])



early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(

    train_dataset.shuffle(10000).batch(512),
    epochs=100,
    validation_data=test_dataset.batch(512),
    verbose=1,
    callbacks=[early_stopping]
)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 128)               124642688 
                                                                 
 dense (Dense)               (None, 16)                2064      
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 124644769 (475.48 MB)
Trainable params: 124644769 (475.48 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/100
69/69 [==============================] - 27s 381ms/step - loss: 0.4718 - accuracy: 0.7321 - val_loss: 0.3063 - val_accuracy: 0.8720
Epoch 2/100
69/69 [==============================] - 26s 381ms/step - loss: 0.2220 - accuracy: 0.9105 - val_loss: 0.2480 - val_acc

#### Wikipedia Corpus

In [8]:
embedding = "https://tfhub.dev/google/Wiki-words-250-with-normalization/2"

hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)

model= tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.summary()

model.compile(optimizer='adam', 
loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
metrics=['accuracy'])

# train_dataset = tf.data.Dataset.from_tensor_slices((train_df['review'], train_df['sentiment_number']))
# test_dataset = tf.data.Dataset.from_tensor_slices((test_df['review'], test_df['sentiment_number']))


early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(

    train_dataset.shuffle(10000).batch(512),
    epochs=100,
    validation_data=test_dataset.batch(512),
    verbose=1,
    callbacks=[early_stopping]
)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 250)               252343750 
                                                                 
 dense_2 (Dense)             (None, 16)                4016      
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 252347783 (962.63 MB)
Trainable params: 252347783 (962.63 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/100


/Users/mazeltan/Desktop/CZ4042_Group_project/env/lib/python3.11/site-packages/keras/src/backend.py:5820: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


69/69 [==============================] - 57s 799ms/step - loss: 0.4458 - accuracy: 0.8186 - val_loss: 0.2898 - val_accuracy: 0.8849
Epoch 2/100
69/69 [==============================] - 78s 1s/step - loss: 0.2146 - accuracy: 0.9190 - val_loss: 0.2527 - val_accuracy: 0.8971
Epoch 3/100
69/69 [==============================] - 77s 1s/step - loss: 0.1388 - accuracy: 0.9523 - val_loss: 0.2701 - val_accuracy: 0.8911
Epoch 4/100
69/69 [==============================] - 82s 1s/step - loss: 0.0925 - accuracy: 0.9721 - val_loss: 0.2996 - val_accuracy: 0.8873
Epoch 5/100
69/69 [==============================] - 74s 1s/step - loss: 0.0610 - accuracy: 0.9847 - val_loss: 0.3460 - val_accuracy: 0.8796
Epoch 6/100
69/69 [==============================] - 71s 1s/step - loss: 0.0384 - accuracy: 0.9926 - val_loss: 0.3929 - val_accuracy: 0.8759
Epoch 7/100
69/69 [==============================] - 72s 1s/step - loss: 0.0242 - accuracy: 0.9964 - val_loss: 0.4401 - val_accuracy: 0.8743


### STANFORD SENTIMENT TREEBANK

In [2]:
data = pytreebank.load_sst("data/SST2-Data/SST2-Data/trainDevTestTrees_PTB/trees/")

In [3]:
out_path = os.path.join("data/sst_{}.txt")

In [4]:
for cat in ['train','test','dev']:
    with open(out_path.format(cat),"w") as file:
        for item in data[cat]:
            file.write("__label__{}\t{}\n".format(
                item.to_labeled_lines()[0][0] +1,
                item.to_labeled_lines()[0][1]
            ))
    
    print("done with {}".format(file))

done with <_io.TextIOWrapper name='data/sst_train.txt' mode='w' encoding='UTF-8'>
done with <_io.TextIOWrapper name='data/sst_test.txt' mode='w' encoding='UTF-8'>
done with <_io.TextIOWrapper name='data/sst_dev.txt' mode='w' encoding='UTF-8'>


In [5]:
train_df2 = pd.read_csv("data/sst_train.txt",sep="\t",header=None,names=['label','text'])
train_df2['label'] = train_df2['label'].str.replace("__label__","")
# df_train['label'] = df_train['label'].astype(int).astype("category")
train_df2['label'] = (train_df2['label'].astype(int) - 1).astype('category')

test_df2 = pd.read_csv("data/sst_test.txt",sep="\t",header=None,names=['label','text'])
test_df2['label'] = test_df2['label'].str.replace("__label__","")
# df_test['label'] = df_test['label'].astype(int).astype("category")
test_df2['label'] = (test_df2['label'].astype(int) - 1).astype('category')

In [10]:
train_df2['text'] = train_df2['text'].apply(strip_html)
train_df2['text'] = train_df2['text'].apply(preprocess_text)
train_df2['text'] = train_df2['text'].apply(remove_punctuation)
train_df2['text'] = train_df2['text'].apply(remove_stopwords)

/var/folders/0s/6y_9xhvn7tx64mnfljyvm01m0000gn/T/ipykernel_81496/584409625.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [11]:
test_df2['text'] = test_df2['text'].apply(strip_html)
test_df2['text'] = test_df2['text'].apply(preprocess_text)
test_df2['text'] = test_df2['text'].apply(remove_punctuation)
test_df2['text'] = test_df2['text'].apply(remove_stopwords)

/var/folders/0s/6y_9xhvn7tx64mnfljyvm01m0000gn/T/ipykernel_81496/584409625.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_df2['text'], train_df2['label']))
test_dataset = tf.data.Dataset.from_tensor_slices((test_df2['text'], test_df2['label']))

In [17]:
embedding = "https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=False)  # You can set trainable to True or False based on your needs

model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(5, activation='softmax'))

model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])


# Batch the datasets
batch_size = 32
train_dataset = train_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    train_dataset,      
    epochs=50,                                 
    validation_data=test_dataset, 
    verbose=1,
    callbacks=[early_stopping]
)


Epoch 1/50
  1/267 [..............................] - ETA: 48s - loss: 1.6192 - accuracy: 0.2812

/Users/mazeltan/Desktop/CZ4042_Group_project/env/lib/python3.11/site-packages/keras/src/backend.py:5729: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


267/267 [==============================] - 1s 1ms/step - loss: 1.4900 - accuracy: 0.3758 - val_loss: 1.7698 - val_accuracy: 0.2864
Epoch 2/50
267/267 [==============================] - 0s 839us/step - loss: 1.4555 - accuracy: 0.3672 - val_loss: 1.6508 - val_accuracy: 0.2937
Epoch 3/50
267/267 [==============================] - 0s 830us/step - loss: 1.3848 - accuracy: 0.3971 - val_loss: 1.6183 - val_accuracy: 0.3027
Epoch 4/50
267/267 [==============================] - 0s 826us/step - loss: 1.3482 - accuracy: 0.4101 - val_loss: 1.5934 - val_accuracy: 0.3190
Epoch 5/50
267/267 [==============================] - 0s 835us/step - loss: 1.3262 - accuracy: 0.4222 - val_loss: 1.5793 - val_accuracy: 0.3267
Epoch 6/50
267/267 [==============================] - 0s 910us/step - loss: 1.3117 - accuracy: 0.4292 - val_loss: 1.5704 - val_accuracy: 0.3353
Epoch 7/50
267/267 [==============================] - 0s 833us/step - loss: 1.3022 - accuracy: 0.4359 - val_loss: 1.5621 - val_accuracy: 0.3439
Epoch

In [13]:
embedding = "https://tfhub.dev/google/Wiki-words-250-with-normalization/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=False)  # You can set trainable to True or False based on your needs

model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(5, activation='softmax'))


model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

batch_size = 32
train_dataset = train_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    train_dataset,      
    epochs=50,                                 
    validation_data=test_dataset, 
    verbose=1,
    callbacks=[early_stopping]
)


Epoch 1/50
  1/267 [..............................] - ETA: 47s - loss: 1.6216 - accuracy: 0.3750

/Users/mazeltan/Desktop/CZ4042_Group_project/env/lib/python3.11/site-packages/keras/src/backend.py:5729: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


267/267 [==============================] - 1s 2ms/step - loss: 1.4737 - accuracy: 0.3923 - val_loss: 1.5824 - val_accuracy: 0.2864
Epoch 2/50
267/267 [==============================] - 0s 636us/step - loss: 1.4771 - accuracy: 0.2973 - val_loss: 1.6254 - val_accuracy: 0.2869
Epoch 3/50
267/267 [==============================] - 0s 587us/step - loss: 1.4134 - accuracy: 0.3865 - val_loss: 1.7825 - val_accuracy: 0.2900
Epoch 4/50
267/267 [==============================] - 0s 592us/step - loss: 1.3947 - accuracy: 0.4106 - val_loss: 1.7264 - val_accuracy: 0.2941
Epoch 5/50
267/267 [==============================] - 0s 582us/step - loss: 1.3771 - accuracy: 0.4096 - val_loss: 1.7463 - val_accuracy: 0.2964
Epoch 6/50
267/267 [==============================] - 0s 1ms/step - loss: 1.3547 - accuracy: 0.4204 - val_loss: 1.7253 - val_accuracy: 0.2968
